In [ ]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [ ]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 参数管理

我们首先关注具有单隐藏层的多层感知机

In [ ]:
torch::nn::Sequential net(torch::nn::Linear(4, 8),
                          torch::nn::ReLU(),
                          torch::nn::Linear(8, 1)
                         );


auto X = torch::rand({2, 4});
printT(X);

printT(net->forward(X));

参数访问

In [ ]:
printT(net[2]->parameters());

// or 

printT(net[2]->name());

auto od = net[2]->named_parameters();

for(auto iter = od.begin(); iter != od.end(); iter++) {
    std::cout << iter->key() << " = " << std::endl;
    std::cout << iter->value() << std::endl;
    std::cout << "<<<--->>>" << std::endl << std::endl;
}

目标参数

In [ ]:
//*** 原教程代码
// print(type(net[2].bias))
// print(net[2].bias)
// print(net[2].bias.data)
    
auto b = od.find("bias");
if(b != nullptr) {
    printT(b->options());
    printT(*b);
    printT(b->data());
}

In [ ]:
auto w = od.find("weight");

if(w != nullptr) {
    printT(w->options());
    printT(w->grad());
}

一次性访问所有参数(同上)

In [ ]:
auto od1 = net->named_parameters();

for(auto iter = od1.begin(); iter != od1.end(); iter++) {
    std::cout << iter->key() << " = " << std::endl;
    std::cout << iter->value() << std::endl;
    std::cout << "<<<--->>>" << std::endl << std::endl;
}

In [ ]:
printT(od1["2.bias"]);

从嵌套块收集参数

In [ ]:
/*design a net*/
struct block1 : torch::nn::Module {
    block1() {
        net = torch::nn::Sequential(torch::nn::Linear(4, 8),
                                    torch::nn::ReLU(),
                                    torch::nn::Linear(8, 4),
                                    torch::nn::ReLU());
    }

    // Implement the Net's algorithm.
    torch::Tensor forward(torch::Tensor x) {
        x = net->forward(x);
        return x;
    }

    // Use one of many "standard library" modules.
    torch::nn::Sequential net{nullptr};
};

In [ ]:
struct block2 : torch::nn::Module {
    block2() {
        net = torch::nn::Sequential(block1(),
                                    block1(),
                                    block1(),
                                    block1()
                                   );
    }

    // Implement the Net's algorithm.
    torch::Tensor forward(torch::Tensor x) {
        x = net->forward(x);
        return x;
    }

    // Use one of many "standard library" modules.
    torch::nn::Sequential net{nullptr};
};

In [ ]:
torch::nn::Sequential rgnet(block2(), torch::nn::Linear(4, 1));
printT(rgnet->forward(X));

In [ ]:
std::cout << rgnet << std::endl;

In [ ]:
for (int i = 0; i < rgnet->size(); i++) {
   printT(rgnet[i]->name()); 
}

对某些块应用不同的初始化方法

In [ ]:
void xavier(torch::nn::Module& m)
{
    if (typeid(m) == typeid(torch::nn::Linear)) {
        auto p = m.named_parameters(false);
        auto w = p.find("weight");
        if (w != nullptr) torch::nn::init::xavier_uniform_(*w);
    }
}

In [ ]:
void init_42(torch::nn::Module& m)
{
    if (typeid(m) == typeid(torch::nn::Linear)) {
        auto p = m.named_parameters(false);
        auto b = p.find("bias");        
        if (b != nullptr) torch::nn::init::constant_(*b, 42);
    }
}     

In [ ]:
net[0]->apply(xavier);
net[2]->apply(init_42);

In [ ]:
auto od3 = net[0]->named_parameters();
auto od4 = net[2]->named_parameters();

printT(od3["weight"]);
printT(od4["bias"]);


自定义初始化

In [ ]:
void my_init(torch::nn::Module& m)
{
    if (typeid(m) == typeid(torch::nn::Linear)) {
        auto p = m.named_parameters(false);
        auto w = p.find("weight");
        if (w != nullptr) torch::nn::init::uniform_(*w, -10, 10);
        auto temp = w->data().abs() >= 5;
        temp.to(torch::kFloat);
        w->data().mm(temp);
    }
}     

In [ ]:
// net[0].weight.data[:] += 1
// net[0].weight.data[0, 0] = 42
// net[0].weight.data[0]

auto od5 = net[0]->named_parameters();
auto w = od5.find("weight");
if (w != nullptr) {
    w->data() += 1;
    w->data()[0][0] = 42;
    printT(*w);
}

参数绑定

In [ ]:
auto shared = torch::nn::Linear(8, 8);
torch::nn::Sequential net2(torch::nn::Linear(4, 8), 
                           torch::nn::ReLU(), 
                           shared, 
                           torch::nn::ReLU(), 
                           shared,
                           torch::nn::ReLU(), 
                           torch::nn::Linear(8, 1)
                          );
net2->forward(X);

In [ ]:
// print(net[2].weight.data[0] == net[4].weight.data[0])
// net[2].weight.data[0, 0] = 100
// print(net[2].weight.data[0] == net[4].weight.data[0])

auto od6 = net2[2]->named_parameters();
auto od7 = net2[4]->named_parameters();
auto w1 = od6.find("weight");
auto w2 = od7.find("weight");

if (w1->data().equal(w2->data())) {
    std::cout << "net[2].weight.data[0] == net[4].weight.data[0]" << std::endl;
}

w1->data()[0][0] = 100;

if (w1->data().equal(w2->data())) {
    std::cout << "net[2].weight.data[0] == net[4].weight.data[0]" << std::endl;
}